In [81]:
# 1) Design model (input, output size, forward pass)
# 2) Construct loss & optimizer
# 3) Training loop
#   - forward pass: compute prediction and loss
#   - backward pass: gradients
#   - update weights

In [82]:
import numpy as np
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

plt.style.use('dark_background')

In [83]:
# 0) Prepare data
bs = datasets.load_breast_cancer()
X, y = bs.data, bs.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# to torch
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

In [84]:
# 1) Design model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [85]:
model = LogisticRegression(n_features)

In [86]:
# 2) Construct loss & optimizer
learning_rate = 0.01
criterion = nn.BCELoss()  # cross entropy
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [87]:
# 3) Training loop
epochs_n = 100

for epoch in range(epochs_n):
    # forward pass
    y_pred = model(X_train)

    # size of error => use cross entropy
    loss = criterion(y_pred, y_train)

    # loss and optimizer
    loss.backward()

    # Backward pass

    # Update weights
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch+1}, loss = {loss.item():.4f}')

Epoch: 10, loss = 0.5203
Epoch: 20, loss = 0.4344
Epoch: 30, loss = 0.3786
Epoch: 40, loss = 0.3394
Epoch: 50, loss = 0.3104
Epoch: 60, loss = 0.2879
Epoch: 70, loss = 0.2698
Epoch: 80, loss = 0.2550
Epoch: 90, loss = 0.2426
Epoch: 100, loss = 0.2320


In [88]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9186
